## Initialize

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "211007"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))

predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics.feather")) 
loghazards = arrow::read_feather(glue("{data_results_path}/loghazards_model_{run}_metabolomics.feather")) %>% pivot_longer(starts_with("logh"), names_to=c("endpoint", "features"), values_to="logh", names_pattern="logh_?(.*)_(.*)$")

In [ ]:
colnames(predictions)

In [ ]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2)))

In [ ]:
logh_NMR = loghazards %>% filter(split=="test") %>% left_join(data %>% select(eid, starts_with("NMR_"), -c(`NMR_measurement_quality_flagged`, `NMR_spectrometer`)) %>% filter(NMR_FLAG==TRUE), by="eid")
logh_NMR_long = logh_NMR %>% pivot_longer(starts_with("NMR_"), names_to="marker", values_to="value")

In [ ]:
library(ggforestplot)

In [ ]:
# Deepexplainer
attributions_de = arrow::read_feather(glue("{data_results_path}/attributions_211026.feather")) %>% mutate(explainer="DeepExplainer")

In [ ]:
attributions = attributions_de

## Attributions by shap

In [ ]:
library("ggbeeswarm")

In [ ]:
run="211007"
name = glue("benchmark_cindex_{run}")
benchmark_cindex_general = read_feather(glue("{data_results_path}/{name}.feather"))  %>% distinct() %>% unite("score", c(module, features), remove=FALSE) %>%  distinct()

In [ ]:
library(ggdist)
perf_order = benchmark_cindex_general %>% filter(module=="DS", features=="Metabolomics") %>% group_by(endpoint) %>% median_qi(cindex) %>% arrange(desc(cindex))
endpoint_order_perf = perf_order$endpoint

In [ ]:
endpoint_order_perf 

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer"
                   
)

In [ ]:
library(ggforestplot)
ng_names = df_NG_biomarker_metadata %>% mutate(metabolite = str_replace_all(tolower(description), " ", "_"))
ng_names %>% sample_n(10)

In [ ]:
ng_names %>% select(group, subgroup) %>% distinct() %>% arrange(group, subgroup)

In [ ]:
library(fuzzyjoin)

In [ ]:
library(fuzzyjoin)
mets1 = attributions %>% select(metabolite) %>% distinct() %>% left_join(ng_names, by = "metabolite")
mets2 = mets1 %>% filter(is.na(name)) %>% select(metabolite) %>% stringdist_left_join(ng_names, by = "metabolite", max_dist = 1) %>% 
    rename(metabolite = metabolite.x) %>% select(-metabolite.y) %>% distinct()
mets3 = mets2 %>% filter(is.na(name)) %>% select(metabolite) %>% stringdist_left_join(ng_names, by = "metabolite", max_dist = 8) %>% 
    rename(metabolite = metabolite.x) %>% select(-metabolite.y) %>% distinct()
mets = bind_rows(mets1 %>% filter(!is.na(name)), mets2 %>% filter(!is.na(name)), mets3)
mets %>% sample_n(5)

In [ ]:
attributions_metadata = attributions %>% left_join(mets %>% select(metabolite, abbreviation, group, subgroup), by="metabolite") %>% mutate(eid=as.integer(as.character(eid)))

In [ ]:
library(gghighlight)

In [ ]:
nmr_real = data %>% select(eid, starts_with("NMR_"), -`NMR_measurement_quality_flagged`, -`NMR_spectrometer`) %>% 
    filter(NMR_FLAG==TRUE) %>% pivot_longer(contains("NMR_"), names_to="metabolite", values_to="met_real") %>% 
    mutate(metabolite = str_remove_all(metabolite, "NMR_"))

In [ ]:
prev_events = data %>% select(eid, starts_with("M_"), -ends_with("_event"), -ends_with("_time")) %>% 
    pivot_longer(contains("M_"), names_to="endpoint", values_to="event") %>% distinct()#%>% 
    #mutate(metabolite = str_remove_all(metabolite, "NMR_"))
prev_events %>% head()

In [ ]:
clean_label = function(label){return(stringr::str_wrap(str_replace_all(label, "_", " "), 20))}

In [ ]:
hrs = loghazards %>% filter(features=="Metabolomics") %>% mutate(hr = exp(logh)) %>% filter(split=="test") %>% select(eid, endpoint, hr)

## Global attributions

In [ ]:
#n_eids = 10000
#eids = (attributions_metadata %>% select(eid) %>% distinct() %>% sample_n(n_eids))$eid
met_order_df = attributions_metadata %>% select(group, subgroup, metabolite, abbreviation) %>% distinct() %>% arrange(group, subgroup, abbreviation) %>% mutate(group_id = as.integer(factor(group)))
met_order = met_order_df$metabolite
abbrev_order = met_order_df$abbreviation
group_order = (met_order_df %>% select(group) %>% distinct())$group
attrib_raw = attributions_metadata %>% #filter(eid %in% eids)  %>% 
    left_join(prev_events, by=c("eid", "endpoint")) %>% 
    ungroup() %>% mutate(metabolite=factor(metabolite, levels=met_order)) %>%
    mutate(abbreviation=factor(abbreviation, levels=abbrev_order))#%>% mutate(shap=raster::clamp(shap, -2, +2))

In [ ]:
subgroup_order = c( 'Amino acids',
                    'Branched-chain amino acids',
                   'Aromatic amino acids',
                   'Fluid balance',
                   'Inflammation',
                    'Fatty acids',
                    'Glycolysis related metabolites',
                    'Ketone bodies',
         
                   'Total lipids',
                    'Cholesterol',
                    'Free cholesterol',
                   'Cholesteryl esters',
                   'Phospholipids',
                   'Triglycerides',
                   'Other lipids',
                   
                    'Lipoprotein particle sizes',
                    'Lipoprotein particle concentrations',
                    'Chylomicrons and extremely large VLDL',
                   'Very large VLDL',
                   'Large VLDL',
                   'Medium VLDL',
                   'Small VLDL',
                   'Very small VLDL',
              
                   'Large LDL',
                   'Medium LDL',
                   'Small LDL',
                    'IDL',
                   'Very large HDL',
                   'Large HDL',
                   'Medium HDL',
                   'Small HDL',
                   'Apolipoproteins'
                  )

In [ ]:
attrib_sample = attrib_raw %>% group_by(endpoint, metabolite, explainer) %>% 
    mutate(shap_quantile=ntile(shap, 100), met_quantile=ntile(met_value, 100)) 

In [ ]:
attrib_sample_mean = attrib_sample %>% ungroup() %>% 
    group_by(endpoint, metabolite, abbreviation, group, subgroup, explainer, shap_quantile) %>% 
    summarise(met_quantile=mean(met_quantile), mean_shap = mean(shap), mean_met=mean(met_value)) 

In [ ]:
library(ggforce)

In [ ]:
temp_global = attrib_sample %>% group_by(endpoint, subgroup, metabolite, abbreviation) %>% summarise(global_shap = sum(abs(shap)))

In [ ]:
top = temp_global %>% ungroup() %>% group_by(endpoint) %>% slice_max(global_shap, n=25) %>% mutate(rank = row_number()) %>% mutate(endpoint=factor(endpoint, levels=endpoint_order)) 
top$endpoint = recode(top$endpoint, !!!endpoint_map)

In [ ]:
options(repr.matrix.max.rows=60, repr.matrix.max.cols=200)
top_print = top %>% select(endpoint, abbreviation, rank) %>% ungroup() %>% pivot_wider(names_from="rank", values_from="abbreviation") %>% arrange(endpoint)
top_print